In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100%|███████████████████████████████████████▉| 331M/331M [00:12<00:00, 26.9MB/s]
100%|████████████████████████████████████████| 331M/331M [00:12<00:00, 28.3MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/kaggle/working/utkface-new.zip",'r')
zip.extractall("/kaggle/working")
zip.close()

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy


In [5]:
folder_path = '/kaggle/working/utkface_aligned_cropped/UTKFace/'

In [7]:

import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy




In [8]:
dataset_dir = folder_path

# Set the batch size
batch_size = 64



In [9]:
#preprocessing

images = []
ages = []
genders = []
count =0
for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(dataset_dir, filename))
        img_array = np.array(img)
        images.append(img_array)
        age, gender, *_ = filename.split("_")
        ages.append(int(age))
        genders.append(int(gender))


In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
ages = np.array(ages)
genders = to_categorical(np.array(genders))  # One-hot encode the gender labels

# Split the dataset into training and validation sets
train_images, val_images, train_ages, val_ages, train_genders, val_genders = train_test_split(
    images, ages, genders, test_size=0.2, random_state=42, stratify=genders
)
print("done")


In [11]:
# Loading the pre-trained ResNet-50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False)

# Freezeing to keep weights
for layer in base_model.layers:
    layer.trainable = False

# Adding our layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
age_pred = Dense(1, activation='linear', name='age_output')(x)
gender_pred = Dense(2, activation='sigmoid', name='gender_output')(x)



In [15]:
from tensorflow.keras.models import Model


In [17]:
# Define the model with the two outputs
model = Model(inputs=base_model.input, outputs=[age_pred, gender_pred])


In [19]:
# Compile the model with two different losses (regression for age and binary classification for gender)
model.compile(optimizer='adam',
              loss={'age_output': 'mean_squared_error', 'gender_output': 'binary_crossentropy'},
              metrics={'age_output': 'mae', 'gender_output': 'accuracy'})



In [20]:
# Summary of the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 2,101,251 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [21]:
# Fine-tune the model
model.fit(
    train_images, {'age_output': train_ages, 'gender_output': train_genders},
    validation_data=(val_images, {'age_output': val_ages, 'gender_output': val_genders}),
    batch_size=batch_size, epochs=30)


Epoch 1/30


I0000 00:00:1729151118.363931      88 service.cc:145] XLA service 0x79ed1c0024c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729151118.364013      88 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  2/297 ━━━━━━━━━━━━━━━━━━━━ 22s 75ms/step - age_output_mae: 30.9057 - gender_output_accuracy: 0.4883 - loss: 1370.3679   

I0000 00:00:1729151124.271131      88 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


297/297 ━━━━━━━━━━━━━━━━━━━━ 50s 123ms/step - age_output_mae: 11.9695 - gender_output_accuracy: 0.7304 - loss: 262.0804 - val_age_output_mae: 8.2410 - val_gender_output_accuracy: 0.8229 - val_loss: 127.6469
Epoch 2/30
297/297 ━━━━━━━━━━━━━━━━━━━━ 26s 88ms/step - age_output_mae: 8.0239 - gender_output_accuracy: 0.8325 - loss: 115.4084 - val_age_output_mae: 8.1812 - val_gender_output_accuracy: 0.8393 - val_loss: 114.8403
Epoch 3/30
297/297 ━━━━━━━━━━━━━━━━━━━━ 26s 89ms/step - age_output_mae: 7.4191 - gender_output_accuracy: 0.8425 - loss: 99.8998 - val_age_output_mae: 7.9931 - val_gender_output_accuracy: 0.8536 - val_loss: 110.3488
Epoch 4/30
297/297 ━━━━━━━━━━━━━━━━━━━━ 41s 89ms/step - age_output_mae: 7.1529 - gender_output_accuracy: 0.8630 - loss: 93.4612 - val_age_output_mae: 7.4043 - val_gender_output_accuracy: 0.8682 - val_loss: 100.4063
Epoch 5/30
297/297 ━━━━━━━━━━━━━━━━━━━━ 26s 88ms/step - age_output_mae: 7.0348 - gender_output_accuracy: 0.8677 - loss: 89.7941 - val_age_output_ma

In [22]:

# Save the fine-tuned model
model.save('model.h5')

In [2]:
import os
print(os.listdir('/kaggle/working/'))

['.virtual_documents']
